In [1]:
import os
import codecs
import pandas as pd
import chardet
import pandas as pd
import numpy as np
import jieba as jb
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import gensim
from gensim import corpora
import pyLDAvis.gensim

In [2]:
def get_encoding(file):
    with open(file, 'rb') as f:
        data = f.read()
        return chardet.detect(data)['encoding']

def loadDataFromFile(filepath):       
    edn=''
    data = pd.DataFrame()
    i=0
    print('开始读取文件,请稍等片刻....')
    for fpath,dirs,fs in os.walk(filepath):
        for file in fs:
            filename = fpath+'/'+file
            encodingname= get_encoding(filename)
            if edn != encodingname:
                edn = encodingname
                #print(encodingname,filename)
            label = [dir for dir in fpath.split('/')][-1]
            content = codecs.open(filename, "r", "ANSI").read()
            content_table = pd.DataFrame({'text':content,'label':label,'file':file},index=[i])
            data = data.append(content_table)
            i+=1
    print('文件读取完成!') 
    return data

df = loadDataFromFile('./data/sogou/')

开始读取文件,请稍等片刻....
文件读取完成!


In [2]:
df = pd.read_csv('./data/sogou2006_2.csv')
df=df[['text','label']]
print(len(df))
df.sample(10)

17909


,text,label
15529,\r\r\n\r\r\n\r\r\n&nbsp; \r\r\n 日本、朝鲜和中国，渐渐从文...,C000023
168,做多依据：\r\r\n一、涨停夺人眼球，多方全面胜利\r\r\n周二的市场依然是多头的天下，...,C000008
4514,复方甘草片和复方甘草含片是大众常用药品，主要用于止咳祛痰。需要注意的是复方甘草类药物含有...,C000013
5035,主持人：很多人比较关心肝脏移植术后有哪些并发症，需要注意哪些问题？\r\r\n 彭志海...,C000013
4121,\r\r\n 日前，据报道，研究显示无名指较短以及耳朵有皱纹等体态特征，可能是身体健康出现...,C000013
10068,\r\r\n 新快报讯 \r\r\n广东省自学考试委员会昨日发出《关于调整我省高等教育自学...,C000020
3842,新华网北京5月1日专电（记者李薇薇）国家知识产权战略制定领导小组办公室组织的“2005年...,C000010
11962,黄华琼、高菁、郁舟，三位普通白领，以1984年中国社会开放之初为起点，他们的经历迥异，但...,C000022
8948,4月28日晚7时，全国假日旅游部际协调会议办公室发布了2006年“五一”黄金周第5号旅游...,C000016
1956,《神州经脉》内地股市在恢复再融资的压力下不跌反升，深沪综指在「五一」长假后连续第二日上扬...,C000008


In [3]:
df['label_id'] = df['label'].factorize()[0]
label_id_df = df[['label', 'label_id']].drop_duplicates().sort_values('label_id').reset_index(drop=True)
label_id_df

,label,label_id
0,C000008,0
1,C000010,1
2,C000013,2
3,C000014,3
4,C000016,4
5,C000020,5
6,C000022,6
7,C000023,7
8,C000024,8


In [22]:
df[df.label_id==3]['text'].values[0]

'&nbsp;足彩06032期王智意甲解盘 \r\r\n\u3000\u30001.卡利亚里VS国际米兰：杯赛对国米的吸引力更大，目前平赔较低，本场在“0”的基础上需适当补“1”。\r\r\n\u3000\u30002.切沃VS佛罗伦萨：主队近来状态出现下滑，客队为冠军杯名额则到了不容有失的阶段，然而澳彩却为本场开出主队低水的平手初盘，看来“紫百合”是不会轻松拿到3分。\r\r\n\u3000\u30003.恩波利VS阿斯科利：庄家仅为主场5连胜的恩波利开出平半浅盘，分明是对该队取胜信心不足的表现。\r\r\n\u3000\u30004.拉齐奥VS帕尔马：主队在球半盘下的赢盘能力并不突出，而且初盘存在刻意看低帕尔马的嫌疑，本场补“1”稳妥。\r\r\n\u3000\u30005.AC米兰VS罗马：米兰很少在对手身上发生连续输盘的事情，首回合米兰输盘，本场可对他们看高半线。\r\r\n\u3000\u30006.帕勒莫VS梅西纳：此战已经无关紧要，初盘显示本场是“3/1”格局。\r\r\n\u3000\u30007.雷吉纳VS尤文图斯：初盘显示了客队的强大，且本场关系到客队夺冠问题，此时尤文图斯可任胆选。\r\r\n\u3000\u30008.特雷维索VS乌迪内斯：乌迪内斯在客场已连续拿下3盘，在状态正佳的时候，澳彩只为其开出平手初盘，想必“乌鸡”的势头会就此中断。足彩06032期王智德甲解盘 \r\r\n\u3000\u30009.科隆VS比勒菲尔德：初盘高开意图明显，庄家引筹码去上盘的嫌疑极大，本场科隆有望不败。\r\r\n\u3000\u300010.拜仁慕尼黑VS多特蒙德：多特蒙德客场至多连赢两盘，本场已到其盘路极限，而且澳彩初盘也有意在冷落主队。\r\r\n\u3000\u300011.汉堡VS不莱梅：半球盘下两队在本赛季尚无平局记录，本场适合选择“3/0”。\r\r\n\u3000\u300012.杜伊斯堡VS美因兹：初盘与欧赔极为不符，庄家有利用主队已经降级的题材诱下美因兹的嫌疑。\r\r\n\u3000\u300013.沙尔克04VS斯图加特：初盘主队水位偏高，目前斯图加特客场盘路有反弹迹象，本场不排除客队抢分可能。\r\r\n\u3000\u300014.沃尔夫斯堡VS凯泽斯劳滕：命悬一线的战役，庄家为本场开出了较高的平局赔率，想必两队有望分出胜负。

In [5]:
df[df.label_id==5]['text'].values[10]

'\u3000\u30004月28日，中纪委驻教育部纪检组副组长刘金平在人民网举行的“规范教育收费”网上座谈会上说，教育部历来都坚持义务教育阶段免收学费、免试、就近入学，不允许招收择校生，也不允许变相收取择校生。但他同时表示，择校问题是目前治理教育乱收费过程中最头疼的问题，是块硬骨头，这块硬骨头不啃问题解决不了。\r\r\n\u3000\u3000这段话反映出了当前治理“择校费”问题的现实困境：一方面，义务教育阶段不允许收“择校费”；另一方面，“择校费”大行其道，目前根本没有解决的迹象和可能。而在每年的这个时节，家长在为孩子寻找一个好学校而四处奔波之际，“择校费”又几乎成了一个伪问题！\r\r\n\u3000\u3000如果你去问正在为孩子择校的家长，“择校费”该不该收？他一般会说，当然不该收！再问，他选择的那所学校的“择校费”不收行不行？他一般则会说，不行！显然，对于家长来说，虽然他们承认“择校费”是不该收的钱，但在现实面前，该交的钱他会一分不少地痛快交出，甚至还担心自己交不上这份钱，甚至还要为获得这个交钱的机会而再交一部分钱，甚至还会和学校合谋如何规避风险使学校免于被查处。前两天，一位房地产大亨说“成为房奴那是活该啊”。照此，或许有人也会说出这样的混话来：交“择校费”是一个愿宰，一个愿挨，家长是活该啊，治“择校费”是多管闲事，多此一举。又有谁知家长面对现实的无奈？学校之所以被“择”，只源于它是优质学校，大家都争着来，僧多粥少，“择校费”就是敲门砖。这个价码就是优质学校与一般学校的差价，就是通过市场手段选择教育资源达到的一种平衡。为了孩子能有一个决定未来命运的好学校，为了赢在起点上，做父母的又有谁不愿意为孩子做出最大的努力？尽管钱、权、关系这些东西，对于普通百姓来说都是稀缺资源。\r\r\n\u3000\u3000更有一些家长认为，“择校费”至少体现了某种公平，普通老百姓还可以靠积攒血汗达到择校目的。如果真的不让收“择校费”，什么名目的都不允许的话，那只剩下一个可能：关系大行其道，腐败十分猖獗。而权力、关系、后门等是许多普通老百姓靠积攒血汗也很难得到的。因而，在某种意义上，这对于家长来说更是一种不幸。\r\r\n\u3000\u3000“择校费”是块硬骨头，如果不知它硬在哪里，或知而不为，无动于衷，那么，“择校费”就真的成了不治之症，或者越治越乱，恶性循环以至无穷

In [6]:
df[df.label_id==8]['text'].values[0]

'\r\r\n\u3000\u3000本报北京5月9日电 \r\r\n董宾、高立英报道：北空某导弹团今天传出喜讯，营参谋长谭正提出的兵器改进方案，使困扰该团和兵器设计厂家三载的兵器重大隐患迎刃而解，赢得了官兵和有关专家的赞扬。\r\r\n\u3000\u3000几年前，该团列装新型防空导弹，在检验性实弹打靶中，几次出现故障，影响了该新型武器系统战斗力的发挥。为彻底查找兵器问题隐患，有关兵器研发厂家多次派专家组进行伴随保障和跟踪观察。然而，由于故障症结难被察觉，问题始终久拖未决。\r\r\n\u3000\u3000在该团前不久的实弹演练时，营参谋长谭正在调试兵器参数过程中，敏锐地察觉到这一问题再次出现。经过分析论证，他果断推定：战车某项计算机程序编排不当，是造成系统作战程序相互冲突和影响的主要原因。为此，他大胆提出了改进方案。有关专家对近百样程序进行抽样检测后，肯定了他的看法。针对他的建议，该团随后采取了应急措施，清除了故障隐患，打出了全面列装后的“满堂红”，长期困扰兵器故障终于得以解决。\r\r\n\u3000\u3000看似偶然却艰辛。据了解，为彻底查找出武器系统隐患，谭正经过大量实践，对兵器性能了然于心，记下了几大本操作心得，先后对几十种可能的原因逐一进行推测、判断和排除，对上万组计算机数据进行统计、采样、筛选和分析，最终使问题顺利得以解决。'

In [7]:
#定义删除除字母,数字，汉字以外的所有符号的函数
def remove_punctuation(line):
    line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(u"[^\u4E00-\u9FA5]")
    line = rule.sub('',line)
    return line
 
#停用词列表
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords  
 
#加载停用词
stopwords = stopwordslist("./data/chineseStopWords.txt")

In [24]:
#删除除字母,数字，汉字以外的所有符号
df['clean_text'] = df['text'].apply(remove_punctuation)
 
#分词，并过滤停用词
df['cut_text'] = df['clean_text'].apply(lambda x: " ".join([w for w in list(jb.cut(x)) if w not in stopwords]))
df[['text','clean_text','cut_text']].head()

,text,clean_text,cut_text
0,本报记者陈雪频实习记者唐翔发自上海\r\r\n 一家刚刚成立两年的网络支付公司，它的目...,本报记者陈雪频实习记者唐翔发自上海一家刚刚成立两年的网络支付公司它的目标是成为市值亿美元的上...,本报记者 陈雪频 实习 记者 唐翔 发自 上海 一家 刚刚 成立 两年 网络 支付 公司 目...
1,证券通：百联股份未来5年有能力保持高速增长\r\r\n\r\r\n 深度报告 ...,证券通百联股份未来年有能力保持高速增长深度报告权威内参来自证券通鉴于公司管理层更迭后的新气象...,证券 通百联 股份 未来 年 能力 保持高速 增长 深度 报告 权威 内参 证券 通 公司 ...
2,5月09日消息快评\r\r\n\r\r\n 深度报告 权威内参 来自“证券通”...,月日消息快评深度报告权威内参来自证券通今日热点五一长假过后人民币再度逼近元关口中间价达证券通...,月 日 消息 快评 深度 报告 权威 内参 证券 通 今日热点 五一 长假 过后 人民币 再...
3,5月09日消息快评\r\r\n\r\r\n 深度报告 权威内参 来自“证券通”...,月日消息快评深度报告权威内参来自证券通今日热点五一长假过后人民币再度逼近元关口中间价达证券通...,月 日 消息 快评 深度 报告 权威 内参 证券 通 今日热点 五一 长假 过后 人民币 再...
4,5月09日消息快评\r\r\n\r\r\n 深度报告 权威内参 来自“证券通”...,月日消息快评深度报告权威内参来自证券通今日热点再融资第一单长电行情论坛申能行情论坛有望拔头筹...,月 日 消息 快评 深度 报告 权威 内参 证券 通 今日热点 再融资 第一 单长 电 行情...


In [9]:
no_features = 1000


tfidf = TfidfVectorizer(max_features=no_features)
tfidf_features = tfidf.fit_transform(df.cut_text)
tfidf_feature_names = tfidf.get_feature_names()


cv = CountVectorizer(max_features=no_features)
cv_features = cv.fit_transform(df.cut_text)
cv_feature_names = cv.get_feature_names()

In [10]:
no_topics = 9

#NMF
nmf_tfidf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf_features)

#LDA
lda_cv = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(cv_features)


d:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [11]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("主题 {} : {}".format(topic_idx,"|".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]])))

no_top_words = 10
print('---------------NMF-tfidf_features 主题-----------------------------------------')
display_topics(nmf_tfidf, tfidf_feature_names, no_top_words)
print()
print('--------------Lda-CountVectorizer_features 主题--------------------------------')
display_topics(lda_cv, cv_feature_names, no_top_words)


---------------NMF-tfidf_features 主题-----------------------------------------
主题 0 : 一个|工作|没有|孩子|知道|生活|现在|问题|一种|这种
主题 1 : 市场|企业|行情|机构|产品|增长|公司|证券|业务|投资
主题 2 : 比赛|球队|联赛|球员|队员|主场|新华社|冠军|对手|中国队
主题 3 : 旅游|游客|旅行社|图库|五一|黄金周|景区|记者|中国|乡村
主题 4 : 第页|一页|工作|选择|老板|职业|网页|找到|职场|喜欢
主题 5 : 考生|专业|招生|考试|学生|学校|录取|志愿|高考|教育
主题 6 : 治疗|患者|医院|疾病|病人|女性|症状|药物|手术|健康
主题 7 : 中国|美国|日本|国家|导弹|军事|作战|飞机|俄罗斯|部队
主题 8 : 公司|股东|股份|有限公司|股权|年月日|董事会|公告|会议|流通股

--------------Lda-CountVectorizer_features 主题--------------------------------
主题 0 : 搜狐|直播员|行情|论坛|机构|美元|文章|市场|分秒|指数
主题 1 : 公司|工作|企业|员工|第页|上海|人才|相关|职位|网页
主题 2 : 股东|公司|会议|年月日|股份|股权|有限公司|投票|本次|董事会
主题 3 : 市场|企业|产品|发展|公司|行业|目前|增长|投资|价格
主题 4 : 比赛|训练|记者|已经|表示|没有|进行|一个|时间|战机
主题 5 : 旅游|专业|考生|学校|招生|记者|录取|考试|志愿|学生
主题 6 : 台湾|职业|管理|大陆|制度|台军|香港|培训|法律|改革
主题 7 : 一个|没有|问题|工作|现在|可能|这种|一种|很多|孩子
主题 8 : 中国|美国|日本|国家|发展|文化|军事|世界|导弹|进行


In [12]:
def predict_topic_by_cv(text):
    txt = remove_punctuation(text)
    txt = [w for w in list(jb.cut(txt)) if w not in stopwords]
    txt = [w for w in txt if len(w)>1]
    txt = [" ".join([w for w in txt])]
    newfeature = cv.transform(txt)
    doc_topic_dist_unnormalized = np.matrix(lda_cv.transform(newfeature))
    doc_topic_dist = doc_topic_dist_unnormalized/doc_topic_dist_unnormalized.sum(axis=1)
    topicIdx = doc_topic_dist.argmax(axis=1)[0,0]
    print('该文档属于:主题 {}'.format(topicIdx))
    print("主题 {} : {}".format(topicIdx,"|".join([cv_feature_names[i] for i in (lda_cv.components_[topicIdx,:]).argsort()[:-no_top_words - 1:-1]])))

In [14]:
predict_topic_by_cv('在台“空军司令部”向美方提出发价申请书新购66架F-16V战斗机之后，台媒又曝出美台军售的另一“重头戏”\
                     美制M1坦克售台的最新进展，台军官员称美国上半年就会宣布向台湾出售这款先进坦克。')


该文档属于:主题 8
主题 8 : 中国|美国|日本|国家|发展|文化|军事|世界|导弹|进行


In [15]:
predict_topic_by_cv('经历此次事件以及指数杀跌后，对于已经进入第二阶段的A股来讲，回归价值，从基本面出发或是大概率事件。\
                                目前资金完成了从高位股切换到低位股，第二阶段主升浪要调整选股，\
                                回避第一浪没有业绩支撑涨幅过高的垃圾股和亏损股，重点关注低位的有业绩和成长的龙头股票！')

该文档属于:主题 3
主题 3 : 市场|企业|产品|发展|公司|行业|目前|增长|投资|价格


In [16]:
predict_topic_by_cv('相关报道称，结束了在泰国的拉练，中国国奥队昨日直飞马来西亚吉隆坡。2020年奥运会男足预赛亚洲区第一阶段比赛\
                    （第四届U23亚洲杯预赛），中国队所在小组的比赛将于22日打响，希丁克率领队伍必须打进第二阶段比赛，\
                    他才能拿到下一期的工作合同')


该文档属于:主题 4
主题 4 : 比赛|训练|记者|已经|表示|没有|进行|一个|时间|战机


In [28]:
text_data = df.cut_text.apply(lambda x:x.split())
#过滤掉单个汉字的词语
text_data = text_data.apply(lambda x:[w for w in x if len(w)>1] )

dictionary = corpora.Dictionary(text_data)

#过滤掉词频小于5次,和词频大于90%的词语
dictionary.filter_extremes(no_below=5, no_above=0.9)

In [37]:
#计数的语料
corpus = [dictionary.doc2bow(text) for text in text_data]

#普通LDA模型
import gensim
no_topics = 9
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = no_topics, id2word=dictionary)

topics = ldamodel.print_topics(num_words=8)
for topic in topics:
    print("主题 %d: " % (topic[0]))
    print(topic[1])
    print()


主题 0: 
0.034*"录取" + 0.032*"志愿" + 0.031*"招生" + 0.024*"专业" + 0.016*"考生" + 0.008*"填报" + 0.008*"本科" + 0.008*"学校"

主题 1: 
0.021*"美国" + 0.020*"中国" + 0.010*"日本" + 0.008*"导弹" + 0.007*"军事" + 0.007*"美军" + 0.007*"作战" + 0.006*"海军"

主题 2: 
0.012*"建设" + 0.011*"发展" + 0.010*"国家" + 0.008*"招聘" + 0.008*"中国" + 0.008*"经济" + 0.006*"军队" + 0.006*"安全"

主题 3: 
0.013*"规定" + 0.010*"公司" + 0.007*"工作" + 0.007*"管理" + 0.006*"执行" + 0.006*"要求" + 0.005*"单位" + 0.005*"人员"

主题 4: 
0.015*"中国" + 0.010*"发展" + 0.009*"企业" + 0.008*"文化" + 0.007*"一个" + 0.006*"人才" + 0.006*"工作" + 0.005*"能力"

主题 5: 
0.008*"一个" + 0.008*"工作" + 0.006*"解放军" + 0.006*"问题" + 0.005*"社会" + 0.005*"学生" + 0.005*"进行" + 0.004*"没有"

主题 6: 
0.024*"台湾" + 0.018*"飞机" + 0.014*"战机" + 0.013*"大陆" + 0.011*"飞行" + 0.011*"航母" + 0.009*"计划" + 0.007*"俄罗斯"

主题 7: 
0.056*"公司" + 0.010*"大臣" + 0.007*"裁员" + 0.007*"年月日" + 0.007*"东海" + 0.007*"会议" + 0.006*"有限公司" + 0.006*"关羽"

主题 8: 
0.012*"一个" + 0.009*"没有" + 0.004*"工作" + 0.004*"孩子" + 0.003*"生活" + 0.003*"知道" + 0.003*"训练" + 0.003*"现在"



In [38]:
#tf-idf语料
tfidf = gensim.models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

#多核并行lda模型
no_topics = 9
tf_idf_lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=no_topics, id2word=dictionary, workers=4)

topics = tf_idf_lda_model.print_topics(num_words=8)
for topic in topics:
    print("主题 %d: " % (topic[0]))
    print(topic[1])
    print()

主题 0: 
0.002*"美国" + 0.002*"军事" + 0.002*"中国" + 0.001*"公司" + 0.001*"日本" + 0.001*"武器" + 0.001*"系统" + 0.001*"考生"

主题 1: 
0.001*"公司" + 0.001*"美国" + 0.001*"毕业生" + 0.001*"中国" + 0.001*"市场" + 0.001*"亿美元" + 0.001*"比赛" + 0.001*"军事"

主题 2: 
0.001*"招生" + 0.001*"面试" + 0.001*"孩子" + 0.001*"公司" + 0.001*"中国" + 0.001*"考生" + 0.001*"教育" + 0.001*"学生"

主题 3: 
0.002*"第页" + 0.002*"战机" + 0.002*"工作" + 0.001*"美国" + 0.001*"伊拉克" + 0.001*"日本" + 0.001*"训练" + 0.001*"中国"

主题 4: 
0.004*"海军" + 0.004*"导弹" + 0.003*"美军" + 0.003*"伊朗" + 0.002*"美国" + 0.002*"俄罗斯" + 0.002*"潜艇" + 0.002*"部队"

主题 5: 
0.004*"伊朗" + 0.002*"朝鲜" + 0.001*"美国" + 0.001*"公司" + 0.001*"军事" + 0.001*"中国" + 0.001*"布什" + 0.001*"太平洋"

主题 6: 
0.003*"飞机" + 0.002*"空军" + 0.002*"作战" + 0.002*"部队" + 0.002*"战斗机" + 0.002*"军队" + 0.002*"军事" + 0.002*"公司"

主题 7: 
0.001*"飞行" + 0.001*"公司" + 0.001*"考生" + 0.001*"雷达" + 0.001*"工作" + 0.001*"中国" + 0.001*"考试" + 0.001*"孩子"

主题 8: 
0.001*"公司" + 0.001*"苏联" + 0.001*"股东" + 0.001*"比赛" + 0.001*"蓝军" + 0.001*"年月日" + 0.001*"股份" + 0.001*"旅游"



In [39]:
lda_display = pyLDAvis.gensim.prepare(tf_idf_lda_model, corpus_tfidf, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [40]:
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)